In [ ]:
from arcgis.learn import *
from arcgis.raster import ImageryLayer
from arcgis.gis import GIS
from arcpy import *
from arcpy.sa import *
#from arcpy.ia import *

#import arcpy
import pandas as pd
import requests

# We will use all hardware cores to try to use parallellism to speed up the training model
import multiprocessing

# Target file path
SourceTile = r"U:\ProjectData\027-15_2018a_4BAND.TIF"

# ImageAnalyst extension is required
arcpy.CheckOutExtension("ImageAnalyst")




The aim of this project was to identify areas of mixed land cover on orthophotography 4-band raster data (RGB + near infrared) using new-generation deep learning implementations.
Deep learning is a type of machine learning with several layers of nonlinear processing which allow users to identify patterns, objects, and pixels through models. It is a significant improvement on previous Machine Learning systems since it does not require vast amounts of training samples produced by expert users. Several ArcGis Pro Deep Learning models support sparse training data with remarkable success. In this project, training data for our models was gathered in intervals of less than 30.

There are several different algorithms and deep learning models that can be employed for image classification.  This project is a tool to carry out image classification of land cover by: preprocessing target data, exporting training samples, training and evaluating deep learning models, and displaying results so a human operator can then choose the best fitting model.

The first stage of the project consists on preprocessing our raster data.  This is to ensure images will be processed successfully with DL models at a later stage.

# 0 - Enabling GPU and Parallelism

In [ ]:
# Instruct ArcGis Pro to use GPU acceleration for faster processing
arcpy.env.processorType = "GPU"

# multiprocessing.cpu_count() provides the number of cores - in our case, 16.  Splitting 
arcpy.env.parallelProcessingFactor = (multiprocessing.cpu_count()/100)*3

# 1 - Preprocessing

The first step in our analysis is to convert our target file into a raster object.  Next, we will preprocess our image to enhance.  This will gurantee that the classification process produces the best possible results.

In [ ]:
Source_Raster = Raster(SourceTile)
Source_Raster

## Sharpening

The first preprocessing step is to smooth the target image using the convolution function, which enhances an image with a sharpening 5 X 5 filter.

In [ ]:
# 14 identifies the 5x5 filter required 

Smooth_Raster = arcpy.sa.Convolution(Source_Raster, 14)
Source_Raster.save()
Source_Raster

## Stretching
The next preprocessing stage consists on enhancing an image by changing properties such as brightness, contrast, and gamma. In this case, we use a Sigmoid stretch as it highlights moderate pixel values while maintaining sufficient contrast in the perimeter. The function used applies a sigmoidal function (an S-shaped curve).

In [ ]:
# "3" indicates the sigmoid strength level. The minimum value is 0, and the maximum value is 6.

Stretched_Raster = arcpy.sa.Stretch(Source_Raster, "Sigmoid", 0, None, None, None, True, None, None, None, None, 3)
Stretched_Raster

## Resampling

Resampling changes the spatial resolution of a raster dataset and sets rules for aggregating or interpolating values across the new pixel size.  In this case, we will resample the image according to our datase spatial resolution (40 cm) and we will use the "Nearest" resampling technique, which is suitable for land cover and discrete data.

In [ ]:
# The new resampled file will be saved in ProjectData.

Str_Output_Raster = r"U:\ProjectData\resampled_output.tif"
arcpy.management.Resample(Stretched_Raster, Str_Output_Raster, 0.4, "Nearest")
Resampled_Raster = Raster(Str_Output_Raster)

## Segmentation

Segmentation allows us to identify objects, features, or segments by grouping adjacent pixels with similar spectral and spatial characteristics. The parameters used are spectral detail = 18, spatial detail = 3, minimum segment size = 25.

In [ ]:
Resampled_Raster = SegmentMeanShift(Resampled_Raster, 18, 3, 25)

## Check number of bands and check they are 8 bit unsigned.

This checks the the number of bands in the code and, if it detects that there are more than 3 bands it will extract the first 3.  This is done as some deep learning algorithms are only able to operate on 3-band raster files.  It assumes a certain RGB order, and that the band to be omitted is the 4th band.  This matches related data and project required but might differt in other environments

In [ ]:
#Get the band count and value type of the preprocessed raster. Create string to output the raster after saving.

Str_Raster_Features_BAND  = arcpy.management.GetRasterProperties(Resampled_Raster, "BANDCOUNT")
Str_Raster_Features_VALUETYPE  = arcpy.management.GetRasterProperties(Resampled_Raster, "VALUETYPE")
Str_Output_Raster = r"U:\ProjectData\ThreeBand8Bit_Raster.tif"

#If bandcount <> 4, bands 1, 2 and 3 are extracted
if (Str_Raster_Features_BAND.getOutput(0) != 3):
    Resampled_Raster = arcpy.ia.ExtractBand(Resampled_Raster, [1, 2, 3])

#Copy into a 8 bit unsigned, 3 band file
arcpy.management.CopyRaster(Resampled_Raster, Str_Output_Raster, "", "", "", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "","")

# 2 - Export Training Data 

The second stept in our pixel classification journey is to export training data gathered using ArcGis Pro according to the Deep Learning model requierd.

In [ ]:
#Definition of values for Export Trainig Data for Deep Learning

# inRaster: the target raster file we want to analyse
inRaster = r"U:\ProjectData\ThreeBand8Bit_Raster.tif"

# out_folder is the folder which will contain samples and labels produced by the exporting process.
out_folder = r"U:\ProjectData\ChipsAndLabels"

# in_training is the feautre class saved from ArcGis Pro
in_training = r"U:\ProjectData\trainingsamples.shp"

#Other parametesr for the export training function.

image_chip_format = "TIFF"
tile_size_x = "256"
tile_size_y = "256"
stride_x= "128"
stride_y= "128"
output_nofeature_tiles= "ONLY_TILES_WITH_FEATURES"
metadata_format= "Classified_Tiles"
start_index = 0
classvalue_field = "Classvalue"

This process uses training samples obtained through ArcGis Pro's **Label Objects For Deep Learning** to create two folders: a folder with our sample images (chips) and a folder with labels which will be fed to our Deep Learning Model.

In [ ]:
ExportTrainingDataForDeepLearning(inRaster, out_folder, in_training, image_chip_format,tile_size_x, tile_size_y,stride_x,
                                  stride_y,output_nofeature_tiles,metadata_format, start_index,classvalue_field)

Next, we prepare the Chips and Labels to be used in a Deep Learning model.  In this funtion, batch_size determines the number of training samples to be processed for training at one time. The default value is 2.  The higher the value, the quicker the processing. Values which exceed the computer capabilities will return out of memory errors.

In [ ]:
data = prepare_data(out_folder, batch_size = 8)

A UNET model is trained used the chips and labels in the data object. Ignore_classes is a critical parameter.  It contains the list of class values on which the model will not incur loss. It requirses a value of 0 for the model to run because the NoData class is mapped to '0'.

In [ ]:
unet = UnetClassifier(data, backbone='resnet34', class_balancing = 'False', mixup= 'False', focal_loss = 'False', ignore_classes=[0], chip_size=224, monitor = 'valid_loss')

The best learning rate for the model is now calculated. The learning rate is the amount of change to the model during each step of the optimization process. It is the most important hyperparameter to tune for a neural network in order to achieve good performance, since it The learning rate hyperparameter controls the rate or speed at which the model learns. The lr_find() function provides the best learning rate for our model.

In [ ]:
best_rate = unet.lr_find()

The fit() function trains the model for the specified number of epochs using the specified learning rates.  This feature is very demanding in terms of resources, and it can take many hours to process. To that effect, we used parallelProcessingFactor() to spread the processing of this function between our system hardware cores (16). Spreading a geoprocessing operation across multiple processes can speed up performance and we found out that the processing time was reduced from 308 minutes to 18.

In [ ]:
unet.fit(20, best_rate)

Metrics Calculation

In [ ]:
unet.per_class_metrics()

Saving the model

In [ ]:
unet.save('ModelXVI', framework='PyTorch', publish=True, gis=None, compute_metrics=True, save_optimizer=True, save_inference_file=True)

Classification

In [ ]:
#THE KEY WAS TO USE THE DLPK FILE INSIDE THE DLPK FILE CREATED BY THE UNET.

model_folder=r"C:\Users\jserr\ModelXVI.dlpk"
inRaster = r"C:\Users\jserr\ThreeBand8Bit_Raster.tif"
ClassifiedRaster = r"C:\Users\jserr\ClassifiedRaster.tif"
out_classified_raster = ClassifyPixelsUsingDeepLearning(inRaster, model_folder)